In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
#from torchvision.models import resnet18, resnet18, ResNet18_Weights
import torch.nn.functional as F

import tqdm.notebook as tqdm
from functools import partial
import time

#from google.colab import files

#import resnet

In [2]:
transform_train = transforms.Compose(
    [transforms.RandomCrop(32, padding=4, fill=128),
     transforms.RandomHorizontalFlip(),
     transforms.RandomRotation(30),
     transforms.ToTensor(),
     transforms.Normalize(
         mean=(0.4914, 0.4822, 0.4465),
         std=(0.2023, 0.1994, 0.2010))]
)

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(
         mean=(0.4914, 0.4822, 0.4465),
         std=(0.2023, 0.1994, 0.2010))]
)

In [3]:
batch_size = 32

trainset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size,
    shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform_test
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size,
    shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)

def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, stride=stride, bias=False)

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, expansion=1, *args, **kwargs):
        super().__init__()
        self.in_channels, self.out_channels, self.stride = in_channels, out_channels, stride
        self.relu = nn.ReLU(inplace=True)
        self.expansion = expansion
        self.blocks = nn.Identity()
        if self.should_apply_downsampling:
            self.downsample = nn.Sequential(
                conv1x1(self.in_channels, self.expanded_channels, stride=stride),
                nn.BatchNorm2d(self.expanded_channels)
            )
        else:
            self.downsample = None        

    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion

    @property
    def should_apply_downsampling(self):
        return self.in_channels != self.expanded_channels or self.stride != 1

    def forward(self, x):
        identity = x
        out = self.blocks(x)
        if self.should_apply_downsampling:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [6]:
class BasicBlock(ResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, *args, **kwargs):
        super().__init__(in_channels, out_channels, stride=stride, expantion=1, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv3x3(in_channels, out_channels, stride),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            conv3x3(out_channels, out_channels),
            nn.BatchNorm2d(out_channels)
        )

In [7]:
class BottleneckBlock(ResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1, *args, **kwargs):
        super().__init__(in_channels, out_channels, stride=stride, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv1x1(in_channels, out_channels, stride),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            conv3x3(out_channels, out_channels),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            conv1x1(out_channels, self.expanded_channels),
            nn.BatchNorm2d(self.expanded_channels)
        )

In [8]:
def upsampleBlock(channel_in, channel_out, kernel_size=3, stride=2):
    return nn.Sequential(
        nn.Conv2d(channel_in, 128, kernel_size=1, stride=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=kernel_size, stride=stride),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128, channel_out, kernel_size=1, stride=1),
        nn.BatchNorm2d(channel_out),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d((1,1))
    )

In [9]:
def downsampleBlock(channel_in, channel_out, stride=2):
    return nn.Sequential(
        nn.Conv2d(channel_in, 128, kernel_size=1, stride=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, stride=stride, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128, channel_out, kernel_size=1, stride=1),
        nn.BatchNorm2d(channel_out),
        nn.ReLU(),
        )

In [10]:
class ResNet(nn.Module):
    def __init__(self, block, layers, n_classes=100, width_decrement=1, *args, **kwargs):
        super().__init__()
        self.in_channels = 64//width_decrement
        self.width_decrement = width_decrement

        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=(3, 3), stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64//width_decrement, block, layers[0])
        self.fc1 = nn.Linear(self.in_channels*block.expansion, n_classes)

        self.layer2 = self._make_layer(128//width_decrement, block, layers[1], stride=2)
        self.fc2 = nn.Linear(self.in_channels*block.expansion, n_classes)

        self.layer3 = self._make_layer(256//width_decrement, block, layers[2], stride=2)
        self.fc3 = nn.Linear(self.in_channels*block.expansion, n_classes)

        self.layer4 = self._make_layer(512//width_decrement, block, layers[3], stride=2)
        self.fc4 = nn.Linear(self.in_channels*block.expansion, n_classes)

        self.upsample1 = upsampleBlock(64//width_decrement, 512//width_decrement)
        self.upsample2 = upsampleBlock(128//width_decrement, 512//width_decrement)
        self.upsample3 = upsampleBlock(256//width_decrement, 512//width_decrement)
        
        self.downsample1 = downsampleBlock(512//width_decrement, 64//width_decrement)
        self.downsample2 = downsampleBlock(512//width_decrement, 128//width_decrement)
        self.downsample3 = downsampleBlock(512//width_decrement, 256//width_decrement)
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        #self.fc = nn.Linear((512 // width_decrement)*block.expansion, n_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        
    def _make_layer(self, out_channels, block_type, num_blocks, stride=1):
        layers = []
        if num_blocks == 0: return nn.Sequential(*layers)
        layers.append(block_type(self.in_channels, out_channels, stride=stride))
        self.in_channels = block_type.expansion * out_channels
        for _ in range(1, num_blocks):
            layers.append(block_type(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        feature_list = []
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)

        out1 = self.upsample1(x)
        feature_list.append(out1.view(out1.size(0), -1))
        out1 = self.downsample1(out1)
        out1 = self.avg_pool(out1)
        out1 = out1.view(out1.size(0), -1)

        x = self.layer2(x)

        out2 = self.upsample2(x)
        feature_list.append(out2.view(out2.size(0), -1))
        out2 = self.downsample2(out2)
        out2 = self.avg_pool(out2)
        out2 = out2.view(out2.size(0), -1)

        x = self.layer3(x)

        out3 = self.upsample3(x)
        feature_list.append(out3.view(out3.size(0), -1))
        out3 = self.downsample3(out3)
        out3 = self.avg_pool(out3)
        out3 = out3.view(out3.size(0), -1)

        x = self.layer4(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        feature_list.append(x)

        out1 = self.fc1(out1)
        out2 = self.fc2(out2)
        out3 = self.fc3(out3)
        x = self.fc4(x)
        return [out1, out2, out3, x], feature_list

In [11]:
def resnet18(path='', pretrained=False, progress=True, **kwargs):
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs) 
    if pretrained:
        model = model.load_state_dict(torch.load(path))
        model.eval()
    return model

In [12]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda', 0)
    
print(type(device), device)

<class 'torch.device'> cuda:0


In [13]:
model18 = resnet18(n_classes=10, width_decrement=2)

In [14]:
model18.to(device)

ResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (relu): ReLU(inplace=True)
      (blocks): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BasicBlock(
      (relu): ReLU(inplace=True)
      (blocks): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     

In [15]:
def KDLoss(x, labels, temperature=3.0):
    log_softmax_outputs = F.log_softmax(x/temperature, dim=1)
    softmax_targets = F.softmax(labels/temperature, dim=1)
    return -(log_softmax_outputs * softmax_targets).sum(dim=1).mean()

In [16]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model18.parameters(), lr=0.0001, weight_decay=5e-4)

In [17]:
def training_loop(n_epochs, network, loss_fn, optimizer, dl_train, dl_test, device, ce_coef=0.3, lambda_coef=0.03):
    train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []
    train_losses_sum , test_losses_sum = [], []
    last_loss = 100
    patience = 3
    trigger_times = 0
    best_acc = 75
    pbar = tqdm.tqdm(range(n_epochs), total=n_epochs, leave=False)
    for epoch in pbar:
        # Переводим сеть в режим обучения
        network.train(True)

        # Итерация обучения сети
        for images, labels in tqdm.tqdm(dl_train, total=len(dl_train), leave=False):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs, features = network(images)
            loss = (1 - ce_coef) * loss_fn(outputs[-1], labels)
            for i in range(len(outputs) - 1):
                loss += (1 - ce_coef) * loss_fn(outputs[i], labels)
                loss += ce_coef * KDLoss(outputs[i], outputs[-1])
                loss += lambda_coef * torch.dist(features[i], features[-1]) 
            loss.backward()
            optimizer.step()
        
        # Оцениваем качество модели каждые 3 итерации
        if epoch % 3 == 0 or epoch == n_epochs - 1:
            network.eval()

            with torch.no_grad():
                # Вычисление качества и функции потерь на обучающей выборке
                train_ac = [0. for _ in range(4)]
                train_l = [0. for _ in range(4)]
                train_l_sum = 0.
                total_train=len(dl_train)
                for images, labels in iter(dl_train):
                    images, labels = images.to(device), labels.to(device)
                    train_outputs, train_features = network(images)
                    train_l[3] += (1 - ce_coef) * loss_fn(train_outputs[-1], labels)
                    train_l_sum += (1 - ce_coef) * loss_fn(train_outputs[-1], labels)
                    train_pred = train_outputs[-1].argmax(axis = 1)
                    train_ac[3] += (train_pred == labels).float().sum() / images.shape[0] * 100.
                    for i, out in enumerate(train_outputs[:-1]):
                        loss_part1 = (1 - ce_coef) * loss_fn(out, labels)
                        loss_part2 = ce_coef * KDLoss(out, train_outputs[-1])
                        loss_part3 = lambda_coef * torch.dist(train_features[i], train_features[-1])
                        train_l[i] += loss_part1
                        train_l[i] += loss_part2
                        train_l[i] += loss_part3
                        train_l_sum += loss_part1 + loss_part2 + loss_part3
                        train_pred = train_outputs[i].argmax(axis = 1)
                        train_ac[i] += (train_pred == labels).float().sum() / images.shape[0] * 100.
                for i in range(4):
                    train_l[i] /= total_train
                    train_ac[i] /= total_train
                train_l_sum /= total_train
                train_accuracies.append(train_ac)
                train_losses.append(train_l)
                train_losses_sum.append(train_l_sum)

                # Вычисление качества и функции потерь на тестовой выборке
                test_ac = [0. for _ in range(4)]
                test_l = [0. for _ in range(4)]
                test_l_sum = 0.
                total_test = len(dl_test)
                for images, labels in iter(dl_test):
                    images, labels = images.to(device), labels.to(device)
                    test_outputs, test_features = network(images)
                    test_l[3] += (1 - ce_coef) * loss_fn(test_outputs[-1], labels)
                    test_l_sum += (1 - ce_coef) * loss_fn(test_outputs[-1], labels)
                    test_pred = test_outputs[-1].argmax(axis = 1)
                    test_ac[3] += (test_pred == labels).float().sum() / images.shape[0] * 100.
                    for i, out in enumerate(test_outputs[:-1]):
                        loss_part1 = (1 - ce_coef) * loss_fn(out, labels)
                        loss_part2 = ce_coef * KDLoss(out, test_outputs[-1])
                        loss_part3 = lambda_coef * torch.dist(test_features[i], test_features[-1])
                        test_l[i] += loss_part1
                        test_l[i] += loss_part2
                        test_l[i] += loss_part3
                        test_l_sum += loss_part1 + loss_part2 + loss_part3
                        test_pred = test_outputs[i].argmax(axis = 1)
                        test_ac[i] += (test_pred == labels).float().sum() / images.shape[0] * 100.
                for i in range(4):
                    test_l[i] /= total_test
                    test_ac[i] /= total_test
                test_l_sum /= total_test
                test_accuracies.append(test_ac)
                test_losses.append(test_l)
                test_losses_sum.append(test_l_sum)

            #early stopping
            if test_losses[-1][0] > last_loss:
                trigger_times += 1
                if trigger_times >= patience:
                    print(f'Early stopping at epoch {epoch}')
                    break
            else:
                trigger_times = 0
            last_loss = test_losses[-1][0]

            #сохранение лучшей модели
            #if test_accuracies[-1] > best_acc:
            #    state_dict = model6.state_dict()
            #    torch.save(state_dict, 'model6_weights.pt')
            #    #files.download('model18_weights.pt')
            #    best_acc = test_accuracies[-1]
            
            pbar.set_description(
                'Loss (Train/Test): {0:.3f}/{1:.3f}.\n \
                Accuracy1, % (Train/Test): {2:.2f}/{3:.2f}\n \
                Accuracy2, % (Train/Test): {4:.2f}/{5:.2f}\n \
                Accuracy3, % (Train/Test): {6:.2f}/{7:.2f}\n \
                Accuracy4, % (Train/Test): {8:.2f}/{9:.2f}\n'.format(
                    train_losses_sum[-1], test_losses_sum[-1], 
                    train_accuracies[-1][0], test_accuracies[-1][0],
                    train_accuracies[-1][1], test_accuracies[-1][1],
                    train_accuracies[-1][2], test_accuracies[-1][2],
                    train_accuracies[-1][3], test_accuracies[-1][3]
                )
            )
            print('Loss (Train/Test): {0:.3f}/{1:.3f}.\n \
                Accuracy1, % (Train/Test): {2:.2f}/{3:.2f}\n \
                Accuracy2, % (Train/Test): {4:.2f}/{5:.2f}\n \
                Accuracy3, % (Train/Test): {6:.2f}/{7:.2f}\n \
                Accuracy4, % (Train/Test): {8:.2f}/{9:.2f}\n'.format(
                    train_losses_sum[-1], test_losses_sum[-1], 
                    train_accuracies[-1][0], test_accuracies[-1][0],
                    train_accuracies[-1][1], test_accuracies[-1][1],
                    train_accuracies[-1][2], test_accuracies[-1][2],
                    train_accuracies[-1][3], test_accuracies[-1][3]
                ))
            
    return train_losses, test_losses, train_accuracies, test_accuracies, train_losses_sum, test_losses_sum

In [18]:
start = time.time()
n_epochs = 600
train_losses, test_losses, train_accs, test_accs, \
train_losses_sum, test_losses_sum = training_loop(
    n_epochs=n_epochs, network=model18, loss_fn=loss_fn, 
    optimizer=optimizer, dl_train=trainloader, dl_test=testloader, device=device
)
ex_time = time.time() - start

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/1563 [00:00<?, ?it/s]

Loss (Train/Test): 7.481/7.336.
                 Accuracy1, % (Train/Test): 41.75/44.31
                 Accuracy2, % (Train/Test): 39.13/42.59
                 Accuracy3, % (Train/Test): 38.25/41.03
                 Accuracy4, % (Train/Test): 29.29/32.39



  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/1563 [00:00<?, ?it/s]

...

  0%|          | 0/1563 [00:00<?, ?it/s]

Loss (Train/Test): 1.474/2.223.
                 Accuracy1, % (Train/Test): 90.36/86.54
                 Accuracy2, % (Train/Test): 90.88/87.81
                 Accuracy3, % (Train/Test): 93.60/87.82
                 Accuracy4, % (Train/Test): 93.69/88.01



  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/1563 [00:00<?, ?it/s]

Loss (Train/Test): 1.439/2.173.
                 Accuracy1, % (Train/Test): 90.35/86.79
                 Accuracy2, % (Train/Test): 91.46/88.02
                 Accuracy3, % (Train/Test): 94.02/88.40
                 Accuracy4, % (Train/Test): 94.14/88.63



In [19]:
state_dict = model18.state_dict()
torch.save(state_dict, 'model18_weights_final.pt')

In [20]:
torch.save(torch.tensor(train_accs).cpu(), 'train_accs18.pt')
#print(train_accs)

torch.save(torch.tensor(test_accs).cpu(), 'test_accs18.pt')
#print(test_accs)

torch.save(torch.tensor(train_losses).cpu(), 'train_losses18.pt')
#print(train_losses)

torch.save(torch.tensor(test_losses).cpu(), 'test_losses18.pt')
#print(test_losses)

torch.save(torch.tensor(train_losses_sum).cpu(), 'train_losses_sum18.pt')

torch.save(torch.tensor(test_losses_sum).cpu(), 'test_losses_sum18.pt')

In [21]:
print(ex_time)

41916.79290676117
